In [1]:
import os

# pip install python-dotenv
from dotenv import load_dotenv

load_dotenv()

TMDB_API_KEY = os.getenv('TMDB_API_KEY')
TMDB_API_TOKEN = os.getenv('TMDB_API_TOKEN')

if TMDB_API_KEY and TMDB_API_TOKEN:
    print('✅API KEY and TOKEN are set!')
else:
    print('❌API KEY and TOKEN 404')

✅API KEY and TOKEN are set!


In [2]:
# -*- coding: utf-8 -*-
"""
한국에서 제작 + 한국 극장 개봉(2|3) 영화만 수집 → CSV 저장(진행률 표시)
기간: 2005-01-01 ~ 2025-06-30
"""

import os, time, datetime as dt
import requests
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv

# ===================== 설정 =====================
START_DATE = "2005-01-01"
END_DATE   = "2025-06-30"

LANG = "ko-KR"
BASE = "https://api.themoviedb.org/3"
SLEEP = 0.05                       # 요청 간 딜레이
TIMEOUT = 30

INCLUDE_ADULT = False              # 성인물 포함하려면 True
INCLUDE_VIDEO = False              # 보통 False 권장
VOTE_COUNT_MIN = 10                # 너무 마이너 작품 제거(원하면 20/50 등으로 조정)

OUT_CSV = "data/processed/tmdb_kr_theatrical_2005_2025.csv"

# ===================== 준비 =====================
load_dotenv()
API_KEY = os.getenv("TMDB_API_KEY")
if not API_KEY:
    raise SystemExit("❌ TMDB_API_KEY가 환경변수(.env)에 없습니다.")

os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)

# ===================== 공통 유틸 =====================
def get_json(url: str, params: dict, tries: int = 3):
    for i in range(tries):
        try:
            r = requests.get(url, params=params, timeout=TIMEOUT)
            if r.status_code in (429, 500, 502, 503, 504):
                time.sleep(min(2**i, 8)); continue
            r.raise_for_status()
            return r.json()
        except requests.RequestException:
            if i == tries - 1:
                raise
            time.sleep(min(2**i, 8))
    raise RuntimeError("Unreachable")

def common_params():
    return {
        "api_key": API_KEY,
        "language": LANG,
        "include_adult": str(INCLUDE_ADULT).lower(),
        "include_video": str(INCLUDE_VIDEO).lower(),
        "with_vote_count.gte": VOTE_COUNT_MIN,
        "with_origin_country": "KR",     # 🇰🇷 제작국 한국
        "region": "KR",                  # 🇰🇷 개봉 지역 한국
        "with_release_type": "2|3",      # 극장 개봉(제한/정식)
        "sort_by": "release_date.asc",   # 한국 개봉일 기준 정렬
        # release_date.gte/lte는 호출 시 주입
    }

# ===================== 1) ID 수집 =====================
def discover_total_pages(date_gte: str, date_lte: str) -> int:
    p = common_params()
    p.update({"release_date.gte": date_gte, "release_date.lte": date_lte, "page": 1})
    js = get_json(f"{BASE}/discover/movie", p)
    return int(js.get("total_pages", 1)), int(js.get("total_results", 0)), js.get("results", [])

def discover_ids_for_range(date_gte: str, date_lte: str, pbar=None):
    """범위를 /discover로 수집. total_pages<=500이면 한 번에, 아니면 호출자가 슬라이스."""
    p = common_params()
    p.update({"release_date.gte": date_gte, "release_date.lte": date_lte, "page": 1})
    first = get_json(f"{BASE}/discover/movie", p)
    total_pages = int(first.get("total_pages", 1))
    ids = [m["id"] for m in first.get("results", [])]
    if pbar: pbar.update(1)

    for page in range(2, min(total_pages, 500) + 1):
        p["page"] = page
        js = get_json(f"{BASE}/discover/movie", p)
        ids.extend([m["id"] for m in js.get("results", [])])
        if pbar: pbar.update(1)
        time.sleep(SLEEP)
    capped = total_pages > 500
    return ids, capped, total_pages

def year_slices(start: str, end: str):
    s = dt.date.fromisoformat(start); e = dt.date.fromisoformat(end)
    for y in range(s.year, e.year + 1):
        y0 = dt.date(y, 1, 1); y1 = dt.date(y, 12, 31)
        if y0 < s: y0 = s
        if y1 > e: y1 = e
        yield y0.isoformat(), y1.isoformat()

def collect_all_ids():
    print("→ ID 수집 범위:", START_DATE, "~", END_DATE)
    tp, tr, first_results = discover_total_pages(START_DATE, END_DATE)
    print(f"   예상 총 페이지: {tp} (총 편수 예상: {tr})")

    if tp <= 500:
        # 전 구간을 한 번에 수집 (페이지 기반 진행바)
        ids = [m["id"] for m in first_results]
        with tqdm(total=tp, desc="Discover pages (all range)", leave=False) as bar:
            bar.update(1)  # page=1 처리 반영
            more_ids, _, _ = discover_ids_for_range(START_DATE, END_DATE, pbar=bar)
            ids = more_ids  # discover_ids_for_range 안에서 page1부터 다시 처리했으므로 그걸 사용
        return sorted(set(ids))

    # 500 초과 → 연 단위 슬라이스 수집 (연 전체 페이지 합산해서 진행바 표시)
    # 먼저 각 연도의 페이지 수를 파악해 합산
    total_pages_sum = 0
    year_meta = []
    for yg, yl in year_slices(START_DATE, END_DATE):
        pages, _, _ = discover_total_pages(yg, yl)
        year_meta.append((yg, yl, pages))
        total_pages_sum += min(pages, 500)
    print(f"   연 단위 슬라이스 진행 (총 페이지 합산: {total_pages_sum})")

    all_ids = set()
    with tqdm(total=total_pages_sum, desc="Discover pages (by year)", leave=False) as bar:
        for yg, yl, pages in year_meta:
            ids_y, _, _ = discover_ids_for_range(yg, yl, pbar=bar)
            all_ids.update(ids_y)
            time.sleep(SLEEP)
    return sorted(all_ids)

# ===================== 2) 상세 수집 =====================
def fetch_detail(mid: int):
    params = {"api_key": API_KEY, "language": LANG}
    try:
        js = get_json(f"{BASE}/movie/{mid}", params)
        time.sleep(SLEEP)
        return js
    except requests.RequestException:
        return None

# ===================== 3) 정규화 → CSV =====================
def parse_genres(val):
    return [x.get("name") for x in val] if isinstance(val, list) else []

def normalize_rows(rows):
    df = pd.json_normalize(rows)

    out = pd.DataFrame({
        "movie_id": df.get("id"),
        "title": df.get("title"),
        "original_title": df.get("original_title"),
        "original_language": df.get("original_language"),
        "release_date": df.get("release_date"),
        "runtime": pd.to_numeric(df.get("runtime"), errors="coerce"),
        "budget": pd.to_numeric(df.get("budget"), errors="coerce"),
        "revenue": pd.to_numeric(df.get("revenue"), errors="coerce"),
        "vote_average": pd.to_numeric(df.get("vote_average"), errors="coerce"),
        "vote_count": pd.to_numeric(df.get("vote_count"), errors="coerce"),
        "popularity": pd.to_numeric(df.get("popularity"), errors="coerce"),
        "genres": df.get("genres").apply(parse_genres) if "genres" in df else [],
        "production_companies": df.get("production_companies").apply(
            lambda xs: [x.get("name") for x in xs] if isinstance(xs, list) else []
        ) if "production_companies" in df else [],
        "production_countries": df.get("production_countries").apply(
            lambda xs: [x.get("iso_3166_1") for x in xs] if isinstance(xs, list) else []
        ) if "production_countries" in df else [],
    })
    dt_series = pd.to_datetime(out["release_date"], errors="coerce")
    out["release_year"]  = dt_series.dt.year
    out["release_month"] = dt_series.dt.month
    return out

# ===================== 메인 =====================
def main():
    print("1) ID 수집 (🇰🇷 제작 ∩ 🇰🇷 극장 2|3)…")
    ids = collect_all_ids()
    print(f"   → 고유 ID 수: {len(ids):,}")

    print("2) 상세 수집 중…")
    rows = []
    with tqdm(total=len(ids), desc="Fetch details", leave=False) as bar:
        for mid in ids:
            d = fetch_detail(mid)
            if d: rows.append(d)
            bar.update(1)

    print("3) 정규화 및 CSV 저장…")
    df = normalize_rows(rows)
    df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
    print(f"✅ 완료: {OUT_CSV} (rows={len(df):,})")

if __name__ == "__main__":
    main()


1) ID 수집 (🇰🇷 제작 ∩ 🇰🇷 극장 2|3)…
→ ID 수집 범위: 2005-01-01 ~ 2025-06-30
   예상 총 페이지: 234 (총 편수 예상: 4672)


KeyboardInterrupt: 

In [ ]:
# -*- coding: utf-8 -*-
"""
2005-01-01 ~ 2025-12-31
KOFIC(KOBIS)에서 '한국 제작 + 국내 극장 개봉' 영화만 후보 단계부터 선별하여
TMDB 스타일 스키마 CSV 생성 (revenue=국내 최종 누적매출, audience_total=최종 누적관객)

출력 열 순서:
movie_id,title,original_title,original_language,release_date,runtime,budget,
revenue,vote_average,vote_count,popularity,genres,production_companies,
production_countries,release_year,release_month,audience_total
"""

import os, time, json, requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from urllib.parse import urlencode
from tqdm import tqdm
from dotenv import load_dotenv

# ===== 사용자 설정 =====
START_DATE = "2005-01-01"
END_DATE   = "2025-12-31"

OUT_DIR        = "./data_processed"
CANDIDATES_CSV = f"{OUT_DIR}/kofic_candidates_kr_2005_2025.csv"   # 한국만 담긴 후보
OUT_CSV        = f"{OUT_DIR}/kofic_domestic_kr_2005_2025.csv"     # 최종 결과

TIMEOUT = 20
RETRY   = 3
SLEEP   = 0.15  # 너무 낮추면 429 위험

# 주간 박스오피스 기반 최종 누적 추정 파라미터
JUMP_STEPS_DAYS     = [0, 14, 28, 56, 84]  # 개봉 후 0/2/4/8/12주
LOCAL_REFINE_RADIUS = 7                    # 마지막 앵커 ±1주 보정
MIN_ABS_DELTA  = 50_000_000               # 매출 증가 최소치(원)
MIN_REL_DELTA  = 0.01                     # 증가율 임계(1%)
PLATEAU_STREAK = 2                        # 2회 연속 정체 → 종료

# ===== 준비 =====
load_dotenv()
KOFIC_KEY = os.getenv("KOFIC_API_KEY") or os.getenv("KOBIS_KEY")
if not KOFIC_KEY:
    raise SystemExit("❌ .env에 KOFIC_API_KEY(=KOBIS_KEY)가 없습니다.")
os.makedirs(OUT_DIR, exist_ok=True)

BASE = "https://www.kobis.or.kr/kobisopenapi/webservice/rest"

def kobis_get(path, params, retry=RETRY):
    q = {"key": KOFIC_KEY, **params}
    url = f"{BASE}{path}.json?{urlencode(q)}"
    last_err = None
    for i in range(retry):
        try:
            r = requests.get(url, timeout=TIMEOUT)
            if r.status_code in (429, 500, 502, 503, 504):
                time.sleep(min(2**i, 8)); continue
            r.raise_for_status()
            data = r.json()
            if isinstance(data, dict) and data.get("faultInfo"):
                fi  = data["faultInfo"]
                raise RuntimeError(f"KOBIS fault: {fi.get('message')} (code={fi.get('errorCode')})\nURL: {url}")
            time.sleep(SLEEP)
            return data
        except Exception as e:
            last_err = e
            if i == retry - 1:
                print("❌ Request failed:", e, "\nURL:", url)
                raise
            time.sleep(min(2**i, 8))
    raise RuntimeError(f"unreachable: {last_err}")

# ===== 공통 유틸 =====
def to_iso_from_yyyymmdd(s):
    if not s: return pd.NA
    s = str(s)
    if len(s) != 8 or not s.isdigit(): return pd.NA
    return f"{s[:4]}-{s[4:6]}-{s[6:]}"

def json_list_str(xs):
    if xs is None: return "[]"
    if isinstance(xs, list):
        return json.dumps(xs, ensure_ascii=False)
    return json.dumps([str(xs)], ensure_ascii=False)

def sunday_of_week(d: date) -> date:
    # 월=0..일=6 → 일요일로 보정
    return d + timedelta(days=(6 - d.weekday()))

def is_korean_production(info):
    nations = [n.get("nationNm") for n in (info.get("nations") or []) if n.get("nationNm")]
    return any(str(n).strip() in {"한국","대한민국","Korea","South Korea","Republic of Korea"} for n in nations)

def in_period_open(info, start=START_DATE, end=END_DATE):
    openDt = info.get("openDt")
    if not openDt: return False
    iso = to_iso_from_yyyymmdd(openDt)
    if pd.isna(iso): return False
    return (iso >= start) and (iso <= end)

# ===== (옵션2) 공통코드에서 ‘한국’ 8자리 코드 조회 =====
# 에러 메시지에 ‘공통코드220310’이 언급되는 경우가 있어 다양한 후보 comCode를 순회 탐색
CODE_TABLE_CANDIDATES = ["220310", "2204", "22003", "2203", "220301", "220201"]

def get_korea_code(verbose=True):
    for cc in CODE_TABLE_CANDIDATES:
        try:
            js = kobis_get("/code/searchCodeList", {"comCode": cc})
        except Exception as e:
            if verbose:
                print(f"  · 공통코드 {cc} 조회 실패: {e}")
            continue

        # 응답 구조 가변 대응
        code_list = (
            (js.get("codeListResult") or {}).get("codeList")
            or js.get("codes")
            or js.get("codeList")
            or []
        )

        found = None
        for c in code_list:
            name_ko = c.get("korNm") or c.get("codeNm") or c.get("korName") or ""
            name_en = (c.get("engNm") or c.get("engName") or "").upper()
            code    = c.get("fullCd") or c.get("code") or c.get("cd") or ""
            if (("한국" in name_ko) or ("대한민국" in name_ko)
                or ("KOREA" in name_en) or ("REPUBLIC OF KOREA" in name_en)):
                if isinstance(code, str) and len(code) == 8 and code.isdigit():
                    found = code; break

        if found:
            if verbose:
                print(f"✅ 한국 코드 발견 (comCode={cc}): {found}")
            return found

    if verbose:
        print("⚠️ 공통코드 테이블에서 한국 코드(8자리)를 찾지 못했습니다. 응답 필터 폴백을 사용합니다.")
    return None

# ===== 1) 후보(연도별) 수집: ‘서버 필터(한국)’ 우선, 실패시 폴백(응답에서 한국만 선별) =====
KR_LABELS = {"한국","대한민국","Korea","South Korea","Republic of Korea"}

def find_candidates_by_year(year: int, kr_code: str | None):
    y = str(year)
    # 우선 ‘연/기간’ 3가지 패턴으로 프로브 → 성공 조합으로 페이징
    base_patterns = [
        {"openStartDt": y,          "openEndDt": y          },
        {"openStartDt": y                                  },
        {"openStartDt": f"{y}0101", "openEndDt": f"{y}1231"},
    ]

    def probe(params):
        js = kobis_get("/movie/searchMovieList", {**params, "itemPerPage": 10, "curPage": 1})
        res = js.get("movieListResult", {}) or {}
        tot = res.get("totCnt")
        return int(tot) if (isinstance(tot, (int,str)) and str(tot).isdigit()) else 0

    chosen = None
    mode = "ANY"

    # 1) repNationCd=한국코드 사용 프로브
    if kr_code:
        for p in base_patterns:
            params = {**p, "repNationCd": kr_code}
            if probe(params) > 0:
                chosen = params
                mode = "KCODE"
                break

    # 2) 실패 시 무필터로 프로브
    if chosen is None:
        for p in base_patterns:
            if probe(p) > 0:
                chosen = p
                mode = "ANY"
                break

    if chosen is None:
        print(f"  · {year}: 후보 0건 (건너뜀)")
        return []

    out, per_page, cur_page, total_seen = [], 100, 1, None
    while True:
        params = {**chosen, "itemPerPage": per_page, "curPage": cur_page}
        js = kobis_get("/movie/searchMovieList", params)
        res = js.get("movieListResult", {}) or {}
        lst = res.get("movieList", []) or []
        tot = res.get("totCnt", None)
        if total_seen is None and tot is not None:
            total_seen = int(tot)

        if not lst: break

        for x in lst:
            mc  = x.get("movieCd")
            rep = (x.get("repNationNm") or "").strip()
            if not mc:
                continue
            if (mode == "KCODE") or (rep in KR_LABELS):
                out.append((mc, x.get("movieNm"), x.get("openDt")))

        # repNationCd를 쓴 경우에는 totCnt에 신뢰
        if (mode == "KCODE") and (total_seen is not None) and (len(out) >= total_seen):
            break

        cur_page += 1
        time.sleep(SLEEP)

    print(f"  · {year}: mode={mode:<6}, collected(K-only)={len(out):>5}")
    return out

def build_candidates(save_path=CANDIDATES_CSV, start=START_DATE, end=END_DATE):
    s_year, e_year = int(start[:4]), int(end[:4])
    print("연도별 후보 수집(서버 한국코드 우선)…")
    kr_code = get_korea_code(verbose=True)
    cands = []
    for y in range(s_year, e_year+1):
        cands.extend(find_candidates_by_year(y, kr_code))
    df = pd.DataFrame(cands, columns=["movieCd","movieNm","openDt"]).drop_duplicates("movieCd")
    df.to_csv(save_path, index=False, encoding="utf-8-sig")
    print(f"📄 후보 저장(한국만): {save_path} (고유 {len(df):,}편)")
    return df

# ===== 2) 주간 박스오피스 기반 누적(매출/관객) 추정 =====
weekly_cache = {}

def weekly_items(target_yyyymmdd: str):
    if target_yyyymmdd in weekly_cache:
        return weekly_cache[target_yyyymmdd]
    js = kobis_get("/boxoffice/searchWeeklyBoxOfficeList",
                   {"targetDt": target_yyyymmdd, "weekGb": "0"})
    items = js.get("boxOfficeResult", {}).get("weeklyBoxOfficeList", []) or []
    weekly_cache[target_yyyymmdd] = items
    return items

def get_acc_from_weekly_by_code(movieCd: str, target_date: date):
    tgt = sunday_of_week(target_date).strftime("%Y%m%d")
    for it in weekly_items(tgt):
        if it.get("movieCd") == movieCd:
            s = pd.to_numeric(it.get("salesAcc"), errors="coerce") or 0
            a = pd.to_numeric(it.get("audiAcc"),  errors="coerce") or 0
            return s, a
    return 0, 0

def intelligent_final_acc(movieCd: str, open_yyyymmdd: str):
    if not open_yyyymmdd or len(str(open_yyyymmdd)) != 8:
        return (pd.NA, pd.NA)
    base = datetime.strptime(open_yyyymmdd, "%Y%m%d").date()

    best_sales, best_audi = 0.0, 0.0
    plateau_cnt, prev = 0, 0.0

    for dd in JUMP_STEPS_DAYS:
        anchor = base + timedelta(days=dd)
        s, a = get_acc_from_weekly_by_code(movieCd, anchor)
        if s > best_sales: best_sales = s
        if a > best_audi:  best_audi  = a

        delta = best_sales - prev
        rel = (delta/prev) if prev > 0 else 1.0
        plateau_cnt = plateau_cnt + 1 if (delta < MIN_ABS_DELTA and rel < MIN_REL_DELTA) else 0
        prev = best_sales

        if plateau_cnt >= PLATEAU_STREAK:
            for d in range(-LOCAL_REFINE_RADIUS, LOCAL_REFINE_RADIUS+1, 7):
                anchor2 = anchor + timedelta(days=d)
                s2, a2 = get_acc_from_weekly_by_code(movieCd, anchor2)
                if s2 > best_sales: best_sales = s2
                if a2 > best_audi:  best_audi  = a2
            break

    return (pd.NA if best_sales == 0 else best_sales,
            pd.NA if best_audi  == 0 else best_audi)

# ===== 3) 상세 → TMDB 스타일 행 빌드 =====
def fetch_movie_info(movieCd: str):
    js = kobis_get("/movie/searchMovieInfo", {"movieCd": movieCd})
    return js.get("movieInfoResult", {}).get("movieInfo", {}) or {}

def to_tmdb_row(info, salesAcc_final, audiAcc_final):
    movieCd   = info.get("movieCd")
    movieNm   = info.get("movieNm")
    movieNmEn = info.get("movieNmEn") or movieNm
    openDt    = info.get("openDt")  # yyyymmdd
    release_date = to_iso_from_yyyymmdd(openDt)

    dt = pd.to_datetime(release_date, errors="coerce") if pd.notna(release_date) else pd.NaT
    release_year  = (dt.year  if pd.notna(dt) else pd.NA)
    release_month = (dt.month if pd.notna(dt) else pd.NA)

    showTm = pd.to_numeric(info.get("showTm"), errors="coerce")
    genres    = [g.get("genreNm")   for g in (info.get("genres")   or []) if g.get("genreNm")]
    companies = [c.get("companyNm") for c in (info.get("companys") or []) if c.get("companyNm")]

    return {
        "movie_id": movieCd,
        "title": movieNm,
        "original_title": movieNmEn,
        "original_language": "ko",
        "release_date": release_date,
        "runtime": showTm,
        "budget": pd.NA,
        "revenue": pd.to_numeric(salesAcc_final),  # 국내 최종 누적매출(원)
        "vote_average": pd.NA,
        "vote_count": pd.NA,
        "popularity": pd.NA,
        "genres": json_list_str(genres),
        "production_companies": json_list_str(companies),
        "production_countries": json_list_str(["KR"]),
        "release_year": release_year,
        "release_month": release_month,
        "audience_total": pd.to_numeric(audiAcc_final),  # 국내 최종 누적관객
    }

def enrich_and_save(candidates_csv=CANDIDATES_CSV, out_csv=OUT_CSV):
    # 후보 로드 or 생성
    if os.path.exists(candidates_csv):
        cand = pd.read_csv(candidates_csv, dtype=str)
        print(f"📄 후보 로드: {candidates_csv} (rows={len(cand):,})")
    else:
        cand = build_candidates()

    rows = []
    with tqdm(total=len(cand), desc="Detail + domestic totals") as bar:
        for _, r in cand.iterrows():
            movieCd = str(r["movieCd"])
            try:
                info = fetch_movie_info(movieCd)
            except Exception:
                bar.update(1); continue

            # 상세에서도 한국 제작 & 기간 내 개봉 확인 (이중 안전망)
            if (not is_korean_production(info)) or (not in_period_open(info)):
                bar.update(1); continue

            try:
                s_final, a_final = intelligent_final_acc(movieCd, info.get("openDt"))
            except Exception:
                s_final, a_final = (pd.NA, pd.NA)

            rows.append(to_tmdb_row(info, s_final, a_final))
            bar.update(1)

    df = pd.DataFrame(rows)
    if not df.empty:
        df["runtime"]        = pd.to_numeric(df["runtime"], errors="coerce")
        df["revenue"]        = pd.to_numeric(df["revenue"], errors="coerce")
        df["audience_total"] = pd.to_numeric(df["audience_total"], errors="coerce")
        df["release_year"]   = pd.to_numeric(df["release_year"], errors="coerce").astype("Int64")
        df["release_month"]  = pd.to_numeric(df["release_month"], errors="coerce").astype("Int64")

    cols = ["movie_id","title","original_title","original_language","release_date","runtime","budget",
            "revenue","vote_average","vote_count","popularity","genres","production_companies",
            "production_countries","release_year","release_month","audience_total"]
    df = df.reindex(columns=cols)

    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"✅ 저장 완료: {out_csv} (rows={len(df):,})")
    print(f"🗃 weekly_cache 크기(앵커 호출 수): {len(weekly_cache):,}")

# ===== 실행 =====
if __name__ == "__main__":
    if not os.path.exists(CANDIDATES_CSV):
        build_candidates()
    enrich_and_save()


📄 후보 로드: ./data_processed/kofic_candidates_kr_2005_2025.csv (rows=2,100)


Detail + domestic totals:   0%|          | 0/2100 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [2]:
# kobis_weekly_to_tmdb_csv.py
# -*- coding: utf-8 -*-
"""
2005-01-01 ~ 2025-12-31
KOBIS(=KOFIC OpenAPI)에서 '주간 박스오피스 Top10에 한 번이라도 등장'한 영화만 선별
→ 한국 제작만 필터 → TMDB 스타일 스키마 CSV 생성
(revenue = 국내 최종 누적매출, audience_total = 국내 최종 누적관객)

출력 열 순서:
movie_id,title,original_title,original_language,release_date,runtime,budget,
revenue,vote_average,vote_count,popularity,genres,production_companies,
production_countries,release_year,release_month,audience_total
"""

import os, time, json, requests
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
from urllib.parse import urlencode
from tqdm import tqdm
from dotenv import load_dotenv

# ========= 기간/경로 설정 =========
START = date(2005, 1, 1)
END   = date(2025,12,31)

OUT_DIR       = "./data_processed"
WEEKLY_PICKLE = f"{OUT_DIR}/kobis_weekly_pool.pkl"      # 주간 스캔 결과 캐시
OUT_PARTIAL   = f"{OUT_DIR}/kobis_weekly_partial.csv"   # 메타 누적(재시작용)
OUT_FINAL     = f"{OUT_DIR}/kobis_weekly_final.csv"     # 최종 TMDB형

# 하루에 처리할 movieInfo 개수(쿼터 따라 조정)
MOVIES_PER_RUN = 800

# 요청/재시도
TIMEOUT = 20
RETRY   = 3
SLEEP   = 0.12

# ========= 인증 =========
load_dotenv()
KOBIS_KEY = os.getenv("KOFIC_API_KEY") or os.getenv("KOBIS_KEY")
BASE = "https://www.kobis.or.kr/kobisopenapi/webservice/rest"
os.makedirs(OUT_DIR, exist_ok=True)

# ========= 공통 HTTP =========
def kobis_get(path, params, retry=RETRY):
    q = {"key": KOBIS_KEY, **params}
    url = f"{BASE}{path}.json?{urlencode(q)}"
    last = None
    for i in range(retry):
        try:
            r = requests.get(url, timeout=TIMEOUT)
            if r.status_code in (429,500,502,503,504):
                time.sleep(min(2**i,8)); continue
            r.raise_for_status()
            data = r.json()
            if data.get("faultInfo"):
                fi = data["faultInfo"]
                raise RuntimeError(f"KOBIS fault: {fi.get('message')} (code={fi.get('errorCode')})\nURL: {url}")
            time.sleep(SLEEP)
            return data
        except Exception as e:
            last = e
            if i == retry-1: raise
            time.sleep(min(2**i,8))
    raise RuntimeError(last)

# ========= 유틸 =========
def sundays(start: date, end: date):
    # start 기준 그 주 일요일(weekGb=0 사용)
    s = start + timedelta(days=(6 - start.weekday()))
    while s <= end:
        yield s
        s += timedelta(days=7)

def to_iso_from_yyyymmdd(s):
    if not s: return pd.NA
    s = str(s)
    return f"{s[:4]}-{s[4:6]}-{s[6:]}" if len(s) >= 8 else pd.NA

def json_list_str(xs):
    if xs is None: return "[]"
    if isinstance(xs, list): return json.dumps(xs, ensure_ascii=False)
    return json.dumps([str(xs)], ensure_ascii=False)

# ========= 1) 주간 Top10 스캔: 등장 영화 + 최종 누적 =========
def weekly_scan():
    """
    주간 박스오피스(Top10) 전체 주차 스캔 → movieCd별 최댓값 누적매출/관객 및 기본 정보 모음
    반환: dict[movieCd] = {max_sales, max_audi, movieNm, openDt, weeks}
    """
    pool = {}  # movieCd -> dict
    for s in tqdm(list(sundays(START, END)), desc="Weekly scan (Top10)"):
        js = kobis_get("/boxoffice/searchWeeklyBoxOfficeList",
                       {"targetDt": s.strftime("%Y%m%d"), "weekGb": "0"})
        items = js.get("boxOfficeResult",{}).get("weeklyBoxOfficeList",[]) or []
        for it in items:
            cd  = it.get("movieCd")
            if not cd: continue
            nm  = it.get("movieNm")
            odt = it.get("openDt")  # yyyymmdd
            salesAcc = pd.to_numeric(it.get("salesAcc"), errors="coerce") or 0
            audiAcc  = pd.to_numeric(it.get("audiAcc"),  errors="coerce") or 0
            rec = pool.get(cd, {"max_sales":0, "max_audi":0, "movieNm":nm, "openDt":odt, "weeks":0})
            rec["max_sales"] = max(rec["max_sales"], salesAcc)
            rec["max_audi"]  = max(rec["max_audi"],  audiAcc)
            rec["weeks"]     = rec["weeks"] + 1
            if not rec.get("movieNm"):  rec["movieNm"] = nm
            if not rec.get("openDt"):   rec["openDt"]  = odt
            pool[cd] = rec
    pd.to_pickle(pool, WEEKLY_PICKLE)
    return pool

# ========= 2) movieInfo 메타(한국 제작 필터) + TMDB 스키마로 변환 =========
def fetch_movie_info(cd: str):
    js = kobis_get("/movie/searchMovieInfo", {"movieCd": cd})
    return js.get("movieInfoResult",{}).get("movieInfo",{}) or {}

def is_korean_production(info) -> bool:
    nations = [n.get("nationNm") for n in (info.get("nations") or []) if n.get("nationNm")]
    return any(str(n).strip() in {"한국","대한민국","Korea","South Korea","Republic of Korea"} for n in nations)

def to_tmdb_row(cd, pool_rec, info):
    movieNm   = info.get("movieNm") or pool_rec.get("movieNm")
    movieNmEn = info.get("movieNmEn") or movieNm
    openDt    = info.get("openDt") or pool_rec.get("openDt")
    iso       = to_iso_from_yyyymmdd(openDt)
    dt        = pd.to_datetime(iso, errors="coerce") if pd.notna(iso) else pd.NaT

    showTm    = pd.to_numeric(info.get("showTm"), errors="coerce")
    genres    = [g.get("genreNm")   for g in (info.get("genres")   or []) if g.get("genreNm")]
    companies = [c.get("companyNm") for c in (info.get("companys") or []) if c.get("companyNm")]

    return {
        "movie_id": cd,
        "title": movieNm,
        "original_title": movieNmEn,
        "original_language": "ko",
        "release_date": iso,
        "runtime": showTm,
        "budget": pd.NA,
        "revenue": float(pool_rec["max_sales"]),       # 국내 최종 누적매출(원)
        "vote_average": pd.NA,
        "vote_count": pd.NA,
        "popularity": pd.NA,
        "genres": json_list_str(genres),
        "production_companies": json_list_str(companies),
        "production_countries": json_list_str(["KR"]),
        "release_year": (dt.year  if pd.notna(dt) else pd.NA),
        "release_month": (dt.month if pd.notna(dt) else pd.NA),
        "audience_total": float(pool_rec["max_audi"]), # 국내 최종 누적관객
    }

# ========= 3) 실행(재시작 가능) =========
def main():
    if not KOBIS_KEY:
        raise SystemExit("❌ .env에 KOFIC_API_KEY(=KOBIS_KEY)가 없습니다.")

    # 1) 주간 스캔(캐시 활용)
    if os.path.exists(WEEKLY_PICKLE):
        pool = pd.read_pickle(WEEKLY_PICKLE)
    else:
        pool = weekly_scan()

    # 주간에 한 번이라도 잡힌 영화만(=누적매출 추적 가능)
    candidates = [cd for cd, rec in pool.items() if rec.get("max_sales",0) > 0]

    # 2) 이미 수집한 partial 읽기(재시작)
    done = set()
    if os.path.exists(OUT_PARTIAL):
        prev = pd.read_csv(OUT_PARTIAL, dtype=str)
        if "movie_id" in prev.columns:
            done = set(prev["movie_id"].astype(str).tolist())

    todo = [cd for cd in candidates if cd not in done]
    if not todo:
        print("✅ 모두 처리됨. 최종 파일만 정리합니다.")
    else:
        batch = todo[:MOVIES_PER_RUN]
        print(f"이번 배치: {len(batch)}편 / 남은 {len(todo)-len(batch)}편")

        rows = []
        for cd in tqdm(batch, desc="movieInfo + merge (KR only)"):
            try:
                info = fetch_movie_info(cd)
                # 한국 제작 필터 + 개봉일(기간 내 안전망)
                if not is_korean_production(info):
                    continue
                odt = info.get("openDt") or pool[cd].get("openDt")
                if not odt or len(str(odt)) != 8:
                    continue
                # 기간 안전망 (열린 날짜 기준)
                y, m, d = int(odt[:4]), int(odt[4:6]), int(odt[6:8])
                opened = date(y,m,d)
                if opened < START or opened > END:
                    continue

                row = to_tmdb_row(cd, pool[cd], info)
                rows.append(row)
            except Exception as e:
                print("skip", cd, e)

        if rows:
            df = pd.DataFrame(rows)
            header = not os.path.exists(OUT_PARTIAL)
            df.to_csv(OUT_PARTIAL, index=False, mode=("w" if header else "a"),
                      header=header, encoding="utf-8-sig")

    # 3) 최종 정리/저장(TMDB 스키마)
    cols = ["movie_id","title","original_title","original_language","release_date","runtime","budget",
            "revenue","vote_average","vote_count","popularity","genres","production_companies",
            "production_countries","release_year","release_month","audience_total"]

    part = pd.read_csv(OUT_PARTIAL, dtype=str) if os.path.exists(OUT_PARTIAL) else pd.DataFrame(columns=cols)
    if not part.empty:
        part["release_year"]   = pd.to_numeric(part["release_year"], errors="coerce").astype("Int64")
        part["release_month"]  = pd.to_numeric(part["release_month"], errors="coerce").astype("Int64")
        part["runtime"]        = pd.to_numeric(part["runtime"], errors="coerce")
        part["revenue"]        = pd.to_numeric(part["revenue"], errors="coerce")
        part["audience_total"] = pd.to_numeric(part["audience_total"], errors="coerce")
        part = part.drop_duplicates(subset=["movie_id"]).sort_values(["release_year","release_month","title"])

    part.reindex(columns=cols).to_csv(OUT_FINAL, index=False, encoding="utf-8-sig")
    print(f"✅ 저장: {OUT_FINAL} (rows={len(part):,})")

if __name__ == "__main__":
    main()


이번 배치: 800편 / 남은 2249편


movieInfo + merge (KR only): 100%|██████████| 800/800 [09:29<00:00,  1.40it/s]

✅ 저장: ./data_processed/kobis_weekly_final.csv (rows=449)


In [9]:
df = pd.read_csv('./data_processed/kobis_weekly_final.csv')
l = df['title'].to_list()
print(l)

['공공의 적 2', '말아톤', '몽정기2', '키다리 아저씨', 'B형 남자친구', '그때 그 사람들', '레드 아이', '제니, 주노', '파송송 계란탁', '마파도', '여자, 정혜', '잠복근무', '달콤한 인생', '댄서의 순정', '엄마', '역전의 명수', '주먹이 운다(Crying Fist)', '남극일기', '안녕, 형아', '연애술사', '혈의 누', '간 큰 가족', '분홍신', '연애의 목적', '여고괴담 4 : 목소리', '천군', '친절한 금자씨', '가발', '박수칠 때 떠나라', '웰컴 투 동막골', '이대로, 죽을 순 없다(Short Time)', '첼로-홍미주 일가 살인사건', '가문의 위기', '강력 3반', '너는 내 운명', '미스터 주부퀴즈왕', '사랑니', '외출', '형사 Duelist', '내 생애 가장 아름다운 일주일', '새드무비', '야수와 미녀', '오로라 공주', '광식이 동생 광태', '나의 결혼원정기', '무영검', '미스터 소크라테스', '사랑해, 말순씨', '소년, 천국에 가다', '6월의 일기', '애인', '왕의 남자', '작업의 정석', '청연', '태풍', '파랑주의보', '사랑을 놓치다', '싸움의 기술', '야수', '투사부일체', '홀리데이', '구세주', '백만장자의 첫사랑', '손님은 왕이다', '음란서생', '흡혈형사 나도열', '데이지', '로망스', '방과후 옥상', '여교수의 은밀한 매력', '청춘만화', '카리스마 탈출기', '달콤, 살벌한 연인', '도마뱀', '마이 캡틴 김대출', '맨발의 기봉이', '사생결단', '연리지', '가족의 탄생', '공필두', '구타유발자들', '국경의 남쪽', '생, 날선생', '짝패', '호로비츠를 위하여', '강적', '모노폴리', '비열한 거리', '아랑', '아치와 씨팍', '착신아리 파이널', '2월 29일 - 어느날 갑자기 첫번째 이야기', '괴물', '네번째 층 - 어느날 갑자기 두번째 이야기', '아파트', '파이스토리', 

In [ ]:
URL='https://search.naver.com/search.naver?query={query}'

titles = ['공공의 적 2', '말아톤', '몽정기2', '키다리 아저씨', 'B형 남자친구', '그때 그 사람들', '레드 아이', '제니, 주노', '파송송 계란탁', '마파도', '여자, 정혜', '잠복근무', '달콤한 인생', '댄서의 순정', '엄마', '역전의 명수', '주먹이 운다(Crying Fist)', '남극일기', '안녕, 형아', '연애술사', '혈의 누', '간 큰 가족', '분홍신', '연애의 목적', '여고괴담 4 : 목소리', '천군', '친절한 금자씨', '가발', '박수칠 때 떠나라', '웰컴 투 동막골', '이대로, 죽을 순 없다(Short Time)', '첼로-홍미주 일가 살인사건', '가문의 위기', '강력 3반', '너는 내 운명', '미스터 주부퀴즈왕', '사랑니', '외출', '형사 Duelist', '내 생애 가장 아름다운 일주일', '새드무비', '야수와 미녀', '오로라 공주', '광식이 동생 광태', '나의 결혼원정기', '무영검', '미스터 소크라테스', '사랑해, 말순씨', '소년, 천국에 가다', '6월의 일기', '애인', '왕의 남자', '작업의 정석', '청연', '태풍', '파랑주의보', '사랑을 놓치다', '싸움의 기술', '야수', '투사부일체', '홀리데이', '구세주', '백만장자의 첫사랑', '손님은 왕이다', '음란서생', '흡혈형사 나도열', '데이지', '로망스', '방과후 옥상', '여교수의 은밀한 매력', '청춘만화', '카리스마 탈출기', '달콤, 살벌한 연인', '도마뱀', '마이 캡틴 김대출', '맨발의 기봉이', '사생결단', '연리지', '가족의 탄생', '공필두', '구타유발자들', '국경의 남쪽', '생, 날선생', '짝패', '호로비츠를 위하여', '강적', '모노폴리', '비열한 거리', '아랑', '아치와 씨팍', '착신아리 파이널', '2월 29일 - 어느날 갑자기 첫번째 이야기', '괴물', '네번째 층 - 어느날 갑자기 두번째 이야기', '아파트', '파이스토리', '한반도', '각설탕', '다세포소녀', '사랑하니까, 괜찮아', '스승의 은혜', '신데렐라', '아이스케키', '예의없는 것들', '원탁의 천사', '천하장사 마돈나', '플라이 대디', '해변의 여인', '가문의 부활 - 가문의 영광3', '구미호 가족', '두뇌유희프로젝트, 퍼즐', '뚝방전설', '라디오 스타', '무도리', '연애, 그 참을 수 없는 가벼움', '우리들의 행복한 시간', '잘 살아보세', '타짜', '가을로', '거룩한 계보', '마음이...', '폭력써클', '그해 여름', '누가 그녀와 잤을까?', '사랑 따윈 필요 없어', '사랑할 때 이야기하는 것들', '애정결핍이 두 남자에게 미치는 영향', '열혈남아', '잔혹한 출근', '해바라기', 'Mr.로빈 꼬시기', '미녀는 괴로워', '싸이보그지만 괜찮아', '올드미스다이어리', '조용한 세상', '조폭마누라3', '중천', '마파도2', '묵공', '언니가 간다', '오래된 정원', '천년여우 여우비', '최강로맨스', '허브', '1번가의 기적', '그놈 목소리', '김관장 대 김관장 대 김관장', '마강호텔', '바람피기 좋은 날', '복면달호', '뷰티풀 선데이', '빼꼼의 머그잔 여행', '수', '쏜다', '이장과 군수', '좋지 아니한가', '극락도 살인사건', '날아라 허동구', '눈부신 날에', '동갑내기 과외하기 레슨 2', '우아한 세계', '천년학', '못말리는 결혼', '밀양', '아들', '전설의 고향', '검은 집', '두 번째 사랑', '황진이', '꽃미남 연쇄 테러 사건', '므이', '해부학 교실', '화려한 휴가', '기담', '내 생애 최악의 남자', '두 사람이다', '디워', '리턴', '만남의 광장', '사랑방 선수와 어머니', '죽어도 해피엔딩', '지금 사랑하는 사람과 살고 있습니까?', '권순분여사 납치사건', '두 얼굴의 여친', '마이 파더', '브라보 마이 라이프', '사랑', '상사부일체', '어머니는 죽지 않는다', '즐거운 인생', '궁녀', '바르게 살자', '어깨 너머의 연인', '엠', '펀치레이디', '행복', '마을금고 연쇄습격사건', '세븐 데이즈', '스카우트', '식객', '열한번째 엄마', '우리 동네', '내 사랑', '색즉시공 시즌2', '싸움', '용의주도 미스신', '기다리다 미쳐', '더 게임', '뜨거운 것이 좋아', '라듸오 데이즈', '무방비도시', '슈퍼맨이었던 사나이', '어린 왕자', '우리 생애 최고의 순간', '원스어폰어타임', '6년째 연애중', '대한이, 민국씨', '마지막 선물', '바보', '추격자', '마이 뉴 파트너', '숙명', '집결호', '허밍', 'GP 506', '가루지기', '도레미파솔라시도', '비스티 보이즈', '삼국지:용의 부활', '날나리 종부전', '서울이 보이냐', '강철중: 공공의 적 1-1', '걸스카우트', '무림여대생', '크로싱', '흑심모녀', '눈에는 눈 이에는 이', '님은 먼 곳에', '잘못된 만남', '좋은 놈, 나쁜 놈, 이상한 놈', '고死 : 피의 중간고사', '다찌마와리', '아기와 나', '멋진 하루', '신기전', '영화는 영화다', '울학교 이티', '트럭', '고고 70', '그 남자의 책 198쪽', '너를 잊지 않을 거야', '모던보이', '미쓰 홍당무', '비몽', '사과', '아내가 결혼했다', '미인도', '서양 골동 양과자점 앤티크', '소년은 울지 않는다', '순정만화', '1724 기방난동사건', '과속스캔들', '달콤한 거짓말', '로맨틱 아일랜드', '쌍화점', '워낭소리', '유감스러운 도시', '마린보이', '작전', '키친', '핸드폰', '슬픔보다 더 슬픈 이야기', '실종', '7급 공무원', '그림자 살인', '똥파리', '박쥐', '우리 집에 왜 왔니', '인사동 스캔들', '김씨표류기', '마더', '잘 알지도 못하면서', '거북이 달린다', '여고괴담 5', '국가대표', '오감도', '차우', '킹콩을 들다', '해운대', '10억', '불신지옥', '소피의 연애 매뉴얼', '요가학원', '국가대표 완결판-못 다한 이야기', '나는 갈매기', '내 사랑 내 곁에', '불꽃처럼 나비처럼', '애자', '이태원 살인사건', '굿모닝 프레지던트', '부산', '정승필 실종사건', '파주', '호우시절', '19-Nineteen', '내 눈에 콩깍지', '바람', '백야행', '집행자', '천국의 우편배달부', '청담보살', '킬 미', '펜트하우스 코끼리', '홍길동의 후예', '걸프렌즈', '시크릿', '여배우들', '전우치', '식객2 : 김치전쟁', '아빠가 여자를 좋아해', '용서는 없다', '웨딩드레스', '주유소 습격사건 2', '하모니', '의형제', '평행이론', '무법자', '비밀애', '아마존의 눈물', '육혈포 강도단', '구르믈 버서난 달처럼', '반가운 살인자', '베스트셀러', '집 나온 남자들', '친정엄마', '폭풍전야', '꿈은 이루어진다', '내 깡패 같은 애인', '대한민국 1%', '시', '하녀', '맨발의 꿈', '방자전', '포화속으로', '고死 두 번째 이야기: 교생실습', '마음이2', '이끼', '파괴된 사나이', '아저씨', '악마를 보았다', '죽이고 싶은', '폐가', '그랑프리', '김복남 살인사건의 전말', '무적자', '방가? 방가!', '시라노; 연애조작단', '울지마, 톤즈', '퀴즈왕', '해결사', '나탈리', '부당거래', '심야의 F.M.', '참을 수 없는', '돌이킬 수 없는', '두 여자', '불량남녀', '이층의 악당', '초능력자', '페스티발', '김종욱 찾기', '라스트 갓파더', '쩨쩨한 로맨스', '헬로우 고스트', '황해', '글러브', '심장이 뛴다', '조선명탐정 : 각시투구꽃의 비밀', '평양성', '그대를 사랑합니다', '만추', '아이들...', '로맨틱 헤븐', '마이 블랙 미니드레스', '사랑이 무서워', '위험한 상견례', '나는 아빠다', '세상에서 가장 아름다운 이별', '수상한 고객들', '적과의 동침', '써니', '체포왕', '헤드', '회초리', '마마', '모비딕', '풍산개', '화이트: 저주의 멜로디', '고양이: 죽음을 보는 두 개의 눈', '고지전', '마당을 나온 암탉', '퀵', '7광구', '블라인드', '최종병기 활', '푸른소금', '가문의 영광4 - 가문의 수난', '도가니', '의뢰인', '챔프', '카운트다운', '통증', '오늘', '오직 그대만', '완득이', '투혼', '히트', 'Mr.아이돌', '너는 펫', '더 킥', '완벽한 파트너', '커플즈', '특수본', '티끌모아 로맨스', '결정적 한방', '마이 웨이', '오싹한 연애', '퍼펙트 게임', '네버엔딩 스토리', '댄싱퀸', '부러진 화살', '원더풀 라디오', '점박이:한반도의 공룡3D', '코알라 키드 : 영웅의 탄생', '페이스메이커', '러브픽션', '범죄와의 전쟁: 나쁜놈들 전성시대', '파파', '하울링', '가비', '건축학개론', '시체가 돌아왔다', '화차', '간기남', '은교', '인류멸망 보고서', '내 아내의 모든 것', '돈의 맛', '코리아']

# 참여자수
'#main_pack > div.sc_new.cs_common_module.case_empasis.color_14._au_movie_content_wrap > div.cm_content_wrap > div > div > div:nth-child(3) > div.scroll_box > div > div > ul > li:nth-child(1) > div > div.area_intro_info > span.area_people'
# 네티즌평점
'#main_pack > div.sc_new.cs_common_module.case_empasis.color_14._au_movie_content_wrap > div.cm_content_wrap > div > div > div:nth-child(3) > div.scroll_box > div > div > ul > li:nth-child(1) > div > div.area_intro_info > span.area_star_number'
# 평론가 UL
'#main_pack > div.sc_new.cs_common_module.case_empasis.color_14._au_movie_content_wrap > div.cm_content_wrap > div > div > div:nth-child(6) > div > ul'

In [82]:
# pip install requests beautifulsoup4 lxml pandas
import os, re, time, json, random
import pandas as pd
import requests
from time import sleep
from bs4 import BeautifulSoup
titles = ['공공의 적 2', '말아톤', '몽정기2', '키다리 아저씨', 'B형 남자친구', '그때 그 사람들', '레드 아이', '제니, 주노', '파송송 계란탁', '마파도', '여자, 정혜', '잠복근무', '달콤한 인생', '댄서의 순정', '엄마', '역전의 명수', '주먹이 운다(Crying Fist)', '남극일기', '안녕, 형아', '연애술사', '혈의 누', '간 큰 가족', '분홍신', '연애의 목적', '여고괴담 4 : 목소리', '천군', '친절한 금자씨', '가발', '박수칠 때 떠나라', '웰컴 투 동막골', '이대로, 죽을 순 없다(Short Time)', '첼로-홍미주 일가 살인사건', '가문의 위기', '강력 3반', '너는 내 운명', '미스터 주부퀴즈왕', '사랑니', '외출', '형사 Duelist', '내 생애 가장 아름다운 일주일', '새드무비', '야수와 미녀', '오로라 공주',]

PPL_SCORE_SEL = '[data-tab="netizen"] span.area_star_number'
# '#main_pack > div.sc_new.cs_common_module.case_empasis.color_16._au_movie_content_wrap > div.cm_content_wrap > div > div > div._content_chart > div.scroll_box > div > div > ul > li:nth-child(1) > div > div.area_intro_info > span.area_star_number'
PPL_COUNT_SEL = '[data-tab="netizen"] span.area_people'
PPL_UL_SEL = 'ul.area_ulist div.area_text_box'

table = []
for title in titles:
    res = requests.get(f"https://search.naver.com/search.naver?query=영화%20{title}%20관람평").text
    soup = BeautifulSoup(res, "html.parser")
    # naver에 관람평 있으면 저장, 없으면 다음 영화로
    try:
        ppl_score = float(soup.select_one(PPL_SCORE_SEL).text)
        ppl_count = int(soup.select_one(PPL_COUNT_SEL).text.replace('명 참여', '').replace(',', ''))
    except:
        continue
    ppl_ul = soup.select(PPL_UL_SEL)
    
    total = 0
    for item in ppl_ul:
        total += float(item.text[-3:])
    
    try:
        avg = total / len(ppl_ul)
    except ZeroDivisionError:
        avg = None
    
    table.append({
        '제목': title, 
        '네티즌점수': ppl_score, 
        '네티즌평가수': ppl_count, 
        '전문가평점평균': avg,
        '전문가수': len(ppl_ul)
    })
    print(title, ppl_score, ppl_count, avg)
    sleep(0.5)

    
df = pd.DataFrame(table)
df

공공의 적 2 7.471 2678 None
몽정기2 3.011 3273 None
키다리 아저씨 7.371 904 None
B형 남자친구 4.891 1150 None
그때 그 사람들 7.441 2396 None
레드 아이 3.241 811 None
제니, 주노 3.661 3483 None
파송송 계란탁 7.351 1011 None
마파도 6.141 1271 4.0
여자, 정혜 7.491 656 None
잠복근무 7.541 1473 None
달콤한 인생 8.841 6913 None
역전의 명수 6.881 782 None
남극일기 5.561 3093 None
연애술사 7.141 1047 None
혈의 누 8.031 3382 None
간 큰 가족 6.871 1113 None
분홍신 5.611 1292 None
연애의 목적 8.121 3612 None
천군 7.121 2666 None
박수칠 때 떠나라 7.771 3855 6.8
웰컴 투 동막골 8.891 10429 8.0
첼로-홍미주 일가 살인사건 4.121 433 2.75
강력 3반 6.471 1635 6.0
너는 내 운명 9.081 6216 6.6
미스터 주부퀴즈왕 8.001 936 5.0
형사 Duelist 5.891 5677 7.166666666666667
내 생애 가장 아름다운 일주일 8.661 3537 6.8
새드무비 6.641 2398 5.0
야수와 미녀 8.691 3083 4.333333333333333
오로라 공주 8.151 2374 5.4


,제목,네티즌점수,네티즌평가수,전문가평점평균,전문가수
0,공공의 적 2,7.471,2678,NaN,0
1,몽정기2,3.011,3273,NaN,0
2,키다리 아저씨,7.371,904,NaN,0
3,B형 남자친구,4.891,1150,NaN,0
4,그때 그 사람들,7.441,2396,NaN,0
5,레드 아이,3.241,811,NaN,0
6,"제니, 주노",3.661,3483,NaN,0
7,파송송 계란탁,7.351,1011,NaN,0
8,마파도,6.141,1271,4.000000,3
9,"여자, 정혜",7.491,656,NaN,0
